<!--NOTEBOOK_HEADER-->
*This notebook contains material from [CBE60499](https://ndcbe.github.io/CBE60499);
content is available [on Github](git@github.com:ndcbe/CBE60499.git).*


<!--NAVIGATION-->
< [2.2 Integer Programs](https://ndcbe.github.io/CBE60499/02.02-IP.html) | [Contents](toc.html) | [2.4 Dynamic Optimization](https://ndcbe.github.io/CBE60499/02.04-DAE.html) ><p><a href="https://colab.research.google.com/github/ndcbe/CBE60499/blob/master/docs/02.03-GDP.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://ndcbe.github.io/CBE60499/02.03-GDP.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>

# 2.3 Generalized Disjunctive Programs

In [1]:
import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    helper.install_idaes()
    helper.install_ipopt()
    helper.install_cbc()
    # helper.download_figures(['feasible.png'])

In [2]:
milp_solver = 'gurobi'

## 2.3.1 Strip Packing Problem

Reference: https://www.minlp.org/library/problem/index.php?i=121&lib=GDP

Aldo Vecchietti (1) and Ignacio Grossmann (2)

(1) INGAR – Instituto de Desarrollo y Diseño – CONICET – UTN, Avellaneda 3657 – Santa Fe ‐ Argentina

(2) Carnegie Mellon University, 5000 Forbes Av. ‐ Pittsburgh, PA ‐ USA

Description: https://www.minlp.org/problems/ver/156/over/Strip-Packing-Overview.pdf

Results: https://www.minlp.org/problems/ver/156/results/Strip-Packing-Results.pdf

### 2.3.1.1 Define model in Pyomo with GDP

In [3]:
'''
Instead of using
# import pyomo.environ as pyo
We can import specific functions/objects
'''
from pyomo.environ import (ConcreteModel, NonNegativeReals, Objective, Param,
                           Set, SolverFactory, TransformationFactory, Var, value)

# Strip-packing example from http://minlp.org/library/lib.php?lib=GDP

# This model packs a set of rectangles without rotation or overlap within a
# strip of a given width, minimizing the length of the strip.

def create_model():
    """Build the strip packing model."""
    
    model = ConcreteModel(name="Rectangles strip packing")
    
    model.rectangles = Set(ordered=True, initialize=[0, 1, 2, 3, 4, 5, 6, 7])

    # Width and Length of each rectangle
    model.rect_width = Param(
        model.rectangles, initialize={0: 3, 1: 3, 2: 2, 3: 2, 4: 3, 5: 5,
                                      6: 7, 7: 7})
    # parameter indexed by each rectangle
    # length means height
    model.rect_length = Param(
        model.rectangles, initialize={0: 4, 1: 3, 2: 2, 3: 2, 4: 3, 5: 3,
                                      6: 4, 7: 4})

    model.strip_width = Param(
        initialize=10, doc="Available width of the strip")

    # upperbound on length (default is sum of lengths of rectangles)
    model.max_length = Param(
        initialize=sum(model.rect_length[i] for i in model.rectangles),
        doc="maximum length of the strip (if all rectangles were arranged "
        "lengthwise)")

    # x (length) and y (width) coordinates of each of the rectangles
    model.x = Var(model.rectangles,
                  bounds=(0, model.max_length),
                  doc="rectangle corner x-position (position across length)")

    def w_bounds(m, i):
        return (0, m.strip_width - m.rect_width[i])
    model.y = Var(model.rectangles,
                  bounds=w_bounds,
                  doc="rectangle corner y-position (position down width)")

    model.strip_length = Var(
        within=NonNegativeReals, doc="Length of strip required.")

    def rec_pairs_filter(model, i, j):
        return i < j
    model.overlap_pairs = Set(
        initialize=model.rectangles * model.rectangles,
        dimen=2, filter=rec_pairs_filter,
        doc="set of possible rectangle conflicts")

    @model.Constraint(model.rectangles)
    def strip_ends_after_last_rec(model, i):
        return model.strip_length >= model.x[i] + model.rect_length[i]

    model.total_length = Objective(expr=model.strip_length,
                                   doc="Minimize length")

    #
    # Insert the no-overlap disjunctions here!
    #
    # YOUR SOLUTION HERE

    return model

### 2.3.1.2 Transform and Solve with Big M Relaxation

First we will create and print the model.

In [4]:
model = create_model()
model.pprint()

2 Set Declarations
    overlap_pairs : set of possible rectangle conflicts
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     2 :    Any :   28 : {(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 4), (3, 5), (3, 6), (3, 7), (4, 5), (4, 6), (4, 7), (5, 6), (5, 7), (6, 7)}
    rectangles : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    8 : {0, 1, 2, 3, 4, 5, 6, 7}

4 Param Declarations
    max_length : maximum length of the strip (if all rectangles were arranged lengthwise)
        Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :    25
    rect_length : Size=8, Index=rectangles, Domain=Any, Default=None, Mutable=False
        Key : Value
          0 :     4
          1 :     3
          2 :     2
          3 :   

            1 Var Declarations
                indicator_var : Size=1, Index=None
                    Key  : Lower : Value : Upper : Fixed : Stale : Domain
                    None :     0 :  None :     1 : False :  True : Binary

            1 Constraint Declarations
                constraint : Size=1, Index=no_overlap_disjuncts[66].constraint_index, Active=True
                    Key : Lower : Body            : Upper : Active
                      1 :  -Inf : y[2] + 2 - y[6] :   0.0 :   True

            1 LogicalConstraint Declarations
                propositions : Size=0, Index=no_overlap_disjuncts[66].propositions_index, Active=True
                    Key : Body : Active

            5 Declarations: indicator_var constraint_index constraint propositions_index propositions
        no_overlap_disjuncts[67] : Active=True
            2 Set Declarations
                constraint_index : Size=1, Index=None, Ordered=Insertion
                    Key  : Dimen : Domain : Size : Member

Next, let's transform and print the model again. The updated model is really big. This is because the transformation factory replaced the disjunctions with many Big M constraints.

In [5]:
# YOUR SOLUTION HERE

model.pprint()

2 Set Declarations
    overlap_pairs : set of possible rectangle conflicts
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     2 :    Any :   28 : {(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 4), (3, 5), (3, 6), (3, 7), (4, 5), (4, 6), (4, 7), (5, 6), (5, 7), (6, 7)}
    rectangles : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    8 : {0, 1, 2, 3, 4, 5, 6, 7}

4 Param Declarations
    max_length : maximum length of the strip (if all rectangles were arranged lengthwise)
        Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :    25
    rect_length : Size=8, Index=rectangles, Domain=Any, Default=None, Mutable=False
        Key : Value
          0 :     4
          1 :     3
          2 :     2
          3 :   

                                no_overlap_disjuncts[56].indicator_var : Size=1, Index=_pyomo_gdp_bigm_reformulation.relaxedDisjuncts[56].localVarReferences.no_overlap_disjuncts[56].indicator_var_index
                                    Key  : Lower : Value : Upper : Fixed : Stale : Domain
                                    None :     0 :  None :     1 : False :  True : Binary

                            1 SetOf Declarations
                                no_overlap_disjuncts[56].indicator_var_index : Dimen=1, Size=1, Bounds=(None, None)
                                    Key  : Ordered : Members
                                    None :   False : UnindexedComponent_set

                            2 Declarations: no_overlap_disjuncts[56].indicator_var_index no_overlap_disjuncts[56].indicator_var

                    3 Declarations: localVarReferences no_overlap_disjuncts[56].constraint_index no_overlap_disjuncts[56].constraint
                _pyomo_gdp_bigm_reformulation.relaxe

                constraint_index : Size=1, Index=None, Ordered=Insertion
                    Key  : Dimen : Domain : Size : Members
                    None :     1 :    Any :    1 :    {1,}
                propositions_index : Size=1, Index=None, Ordered=Insertion
                    Key  : Dimen : Domain : Size : Members
                    None :    -- :    Any :    0 :      {}

            1 Var Declarations
                indicator_var : Size=1, Index=None
                    Key  : Lower : Value : Upper : Fixed : Stale : Domain
                    None :     0 :  None :     1 : False :  True : Binary

            1 Constraint Declarations
                constraint : Size=1, Index=no_overlap_disjuncts[6].constraint_index, Active=True
                    Key : Lower : Body            : Upper : Active
                      1 :  -Inf : y[0] + 3 - y[2] :   0.0 :  False

            1 LogicalConstraint Declarations
                propositions : Size=0, Index=no_overlap_disjuncts[6].

                constraint_index : Size=1, Index=None, Ordered=Insertion
                    Key  : Dimen : Domain : Size : Members
                    None :     1 :    Any :    1 :    {1,}
                propositions_index : Size=1, Index=None, Ordered=Insertion
                    Key  : Dimen : Domain : Size : Members
                    None :    -- :    Any :    0 :      {}

            1 Var Declarations
                indicator_var : Size=1, Index=None
                    Key  : Lower : Value : Upper : Fixed : Stale : Domain
                    None :     0 :  None :     1 : False :  True : Binary

            1 Constraint Declarations
                constraint : Size=1, Index=no_overlap_disjuncts[79].constraint_index, Active=True
                    Key : Lower : Body            : Upper : Active
                      1 :  -Inf : y[5] + 5 - y[3] :   0.0 :  False

            1 LogicalConstraint Declarations
                propositions : Size=0, Index=no_overlap_disjuncts[79

Finally, we'll solve the model and examine t

In [6]:
#
# Solve and print the solution
#
SolverFactory(milp_solver).solve(model, tee=True)
for i in model.rectangles:
    print("Rectangle %s: (%s, %s)" % (i, value(model.x[i]), value(model.y[i])))
model.total_length.display()

Using license file /Users/adowling/gurobi.lic
Academic license - for non-commercial use only
Read LP format model from file /var/folders/xy/24xvnyss36v3d8mw68tygxdw0000gp/T/tmpwz_0ryap.pyomo.lp
Reading time = 0.00 seconds
x130: 149 rows, 130 columns, 465 nonzeros
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 149 rows, 130 columns and 465 nonzeros
Model fingerprint: 0x8c44476c
Variable types: 18 continuous, 112 integer (112 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 15.0000000
Presolve removed 10 rows and 10 columns
Presolve time: 0.01s
Presolved: 139 rows, 120 columns, 434 nonzeros
Variable types: 17 continuous, 103 integer (103 binary)

Root relaxation: objective 8.000000e+00, 55 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj

### 2.3.1.3 Transform and Solve with Convex Hull Relaxation

We will repeat the procedure above but without printing the model.

In [7]:
model = create_model()

# YOUR SOLUTION HERE

#
# Solve and print the solution
#
SolverFactory(milp_solver).solve(model, tee=True)
for i in model.rectangles:
    print("Rectangle %s: (%s, %s)" % (i, value(model.x[i]), value(model.y[i])))
model.total_length.display()

Using license file /Users/adowling/gurobi.lic
Academic license - for non-commercial use only
Read LP format model from file /var/folders/xy/24xvnyss36v3d8mw68tygxdw0000gp/T/tmpbgzmt60k.pyomo.lp
Reading time = 0.00 seconds
x578: 709 rows, 578 columns, 1921 nonzeros
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 709 rows, 578 columns and 1921 nonzeros
Model fingerprint: 0x328986a1
Variable types: 466 continuous, 112 integer (112 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 4e+00]
Presolve removed 76 rows and 64 columns
Presolve time: 0.00s
Presolved: 633 rows, 514 columns, 1716 nonzeros
Variable types: 411 continuous, 103 integer (103 binary)
Found heuristic solution: objective 25.0000000

Root relaxation: objective 8.000000e+00, 189 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl 

<!--NAVIGATION-->
< [2.2 Integer Programs](https://ndcbe.github.io/CBE60499/02.02-IP.html) | [Contents](toc.html) | [2.4 Dynamic Optimization](https://ndcbe.github.io/CBE60499/02.04-DAE.html) ><p><a href="https://colab.research.google.com/github/ndcbe/CBE60499/blob/master/docs/02.03-GDP.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://ndcbe.github.io/CBE60499/02.03-GDP.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>